# Importar librerias y dataset

In [6]:
import pandas as pd
import re
import warnings
warnings.filterwarnings("ignore")

In [78]:
hoteles = pd.read_csv("../raw_data/Google Maps/reviews-estados/hoteles_crudo.csv", sep="|")

In [7]:
reviews = pd.read_csv("../raw_data/Google Maps/reviews-estados/reviews_hoteles_crudo.csv", sep="|")

# Nulos y duplicados

## Hoteles

In [79]:
hoteles.isna().mean()

name                0.000130
address             0.003506
gmap_id             0.000000
description         0.774026
latitude            0.000000
longitude           0.000000
category            0.000000
avg_rating          0.000000
num_of_reviews      0.000000
price               1.000000
hours               0.995325
MISC                0.913636
state               0.995195
relative_results    0.213247
url                 0.000000
dtype: float64

In [80]:
borrar_columnas = ["description", "price", "hours", "MISC", "relative_results", "url"]
hoteles.drop(columns = borrar_columnas, inplace = True)

In [81]:
hoteles.dropna(subset = ["name", "address"], inplace=True)

In [82]:
hoteles.dtypes

name               object
address            object
gmap_id            object
latitude          float64
longitude         float64
category           object
avg_rating        float64
num_of_reviews      int64
state              object
dtype: object

In [16]:
hoteles.shape

(7672, 9)

In [84]:
hoteles.duplicated(subset='name').sum()

571

In [85]:
hoteles.drop_duplicates(subset='name', inplace=True) 

## Reviews

In [20]:
reviews.isna().mean()

user_id             0.000000
name_x              0.000000
time                0.000000
rating              0.000000
text                0.375033
pics                0.965694
resp                0.903602
gmap_id             0.000000
name_y              0.000000
address             0.000265
description         0.440690
latitude            0.000000
longitude           0.000000
category            0.000000
avg_rating          0.000000
num_of_reviews      0.000000
price               1.000000
hours               0.999510
MISC                0.740711
state               0.999510
relative_results    0.027469
url                 0.000000
dtype: float64

In [21]:
borrar_columnas = ["pics", "resp"]
reviews.drop(columns=borrar_columnas, inplace=True)

In [30]:
reviews.dropna(subset = ["text"], inplace=True)

In [33]:
reviews.duplicated().sum()

2408

In [34]:
reviews.drop_duplicates(inplace=True) 

In [32]:
reviews.dtypes

user_id    object
name_x     object
time        int64
rating      int64
text       object
gmap_id    object
dtype: object

In [36]:
reviews.shape

(44757, 6)

# Normalización

## Tablas dimensionales

### Business, category y state

In [88]:
hoteles.head()

,name,address,gmap_id,latitude,longitude,category,avg_rating,num_of_reviews,state
0,Basecamp Guesthouse,"Basecamp Guesthouse, 49010 SE Middle Fork Rd, ...",0x54907fca2751e187:0x3d6ae31ac18da483,47.471332,-121.686928,['Hotel'],5.0,8,NaN
1,Sugar River Loft,"Sugar River Loft, 929 W Exchange St, Brodhead,...",0x88062b28cb53ef6d:0x94df72568fddf391,42.619862,-89.377890,['Hotel'],4.7,8,NaN
2,Meraki Lodge,"Meraki Lodge, 223 Meraki Ln, Manson, WA 98831",0x549b9d4ad8f113d5:0x8258a10cca866a62,47.887653,-120.120376,"['Hotel', 'Group accommodation', 'Lodge', 'Mot...",4.9,15,NaN
3,View House at Lake Chelan,"View House at Lake Chelan, 245 Summerset Blvd,...",0x549b9db4d73626a3:0x348e8f7106d3be41,47.888291,-120.124120,['Hotel'],4.4,8,NaN
4,Station Lakefront Rental Home,"Station Lakefront Rental Home, 8188 Plum Lake ...",0x4d55cee0af102c25:0x752b313677a78e1c,45.998720,-89.490867,['Resort hotel'],4.7,3,NaN


In [123]:
business = hoteles.reset_index(drop=True)

In [124]:
business = business.reset_index()
business.rename(columns={'index': 'business_id', 'name': 'business_name'}, inplace=True)
business.head()

,business_id,business_name,address,gmap_id,latitude,longitude,category,avg_rating,num_of_reviews,state
0,0,Basecamp Guesthouse,"Basecamp Guesthouse, 49010 SE Middle Fork Rd, ...",0x54907fca2751e187:0x3d6ae31ac18da483,47.471332,-121.686928,['Hotel'],5.0,8,NaN
1,1,Sugar River Loft,"Sugar River Loft, 929 W Exchange St, Brodhead,...",0x88062b28cb53ef6d:0x94df72568fddf391,42.619862,-89.377890,['Hotel'],4.7,8,NaN
2,2,Meraki Lodge,"Meraki Lodge, 223 Meraki Ln, Manson, WA 98831",0x549b9d4ad8f113d5:0x8258a10cca866a62,47.887653,-120.120376,"['Hotel', 'Group accommodation', 'Lodge', 'Mot...",4.9,15,NaN
3,3,View House at Lake Chelan,"View House at Lake Chelan, 245 Summerset Blvd,...",0x549b9db4d73626a3:0x348e8f7106d3be41,47.888291,-120.124120,['Hotel'],4.4,8,NaN
4,4,Station Lakefront Rental Home,"Station Lakefront Rental Home, 8188 Plum Lake ...",0x4d55cee0af102c25:0x752b313677a78e1c,45.998720,-89.490867,['Resort hotel'],4.7,3,NaN


In [125]:
valid_state_codes = ["AL", "AK", "AZ", "AR", "CA", "CO", "CT", "DC", "DE", "FL", "GA", "HI", "ID", "IL", "IN", "IA", "KS", 
                    "KY", "LA", "ME", "MD", "MA", "MI", "MN", "MS", "MO", "MT", "NE", "NV", "NH", "NJ", "NM", "NY", 
                    "NC", "ND", "OH", "OK", "OR", "PA", "RI", "SC", "SD", "TN", "TX", "UT", "VT", "VA", "WA", "WV", "WI", "WY"]

In [132]:
def extract_state_code(address):
    # Expresión regular para buscar códigos de estado válidos
    regex_pattern = r"\b(" + "|".join(valid_state_codes) + r")\b"

    # Busca el código de estado en la dirección usando regex
    match = re.search(regex_pattern, address)

    # Devuelve el código de estado si se encuentra, o None si no se encuentra
    return match.group(0) if match else None

# Crea una nueva columna "state" aplicando la función a la columna "address"

business['state'] = business['address'].apply(extract_state_code)

In [134]:
business.isna().sum()

business_id        0
business_name      0
address            0
gmap_id            0
latitude           0
longitude          0
category           0
avg_rating         0
num_of_reviews     0
state             31
dtype: int64

In [135]:
business.dropna(subset="state", inplace=True)

In [203]:
business.head()

,business_id,business_name,address,gmap_id,latitude,longitude,category,avg_rating,num_of_reviews,state
0,0,Basecamp Guesthouse,"Basecamp Guesthouse, 49010 SE Middle Fork Rd, ...",0x54907fca2751e187:0x3d6ae31ac18da483,47.471332,-121.686928,[Hotel],5.0,8,WA
1,1,Sugar River Loft,"Sugar River Loft, 929 W Exchange St, Brodhead,...",0x88062b28cb53ef6d:0x94df72568fddf391,42.619862,-89.377890,[Hotel],4.7,8,WI
2,2,Meraki Lodge,"Meraki Lodge, 223 Meraki Ln, Manson, WA 98831",0x549b9d4ad8f113d5:0x8258a10cca866a62,47.887653,-120.120376,"[Hotel, Group accommodation, Lodge, Motel]",4.9,15,WA
3,3,View House at Lake Chelan,"View House at Lake Chelan, 245 Summerset Blvd,...",0x549b9db4d73626a3:0x348e8f7106d3be41,47.888291,-120.124120,[Hotel],4.4,8,WA
4,4,Station Lakefront Rental Home,"Station Lakefront Rental Home, 8188 Plum Lake ...",0x4d55cee0af102c25:0x752b313677a78e1c,45.998720,-89.490867,[Resort hotel],4.7,3,WI


Usamos explode() para expandir las listas de la columna category, los nuevos registros de categoria conservan su respectivo index de la anterior tabla, pero al haber creado la columna business_id con valores iguales al index de dicha tabla, los valores de categoria ahora coinciden correctamente con el id de los hoteles.

In [136]:
import ast

business['category'] = business['category'].apply(ast.literal_eval)

In [137]:
business_category = business.explode('category')["category"].reset_index()
business_category.rename(columns={'index': 'business_id'}, inplace=True)
business_category.head()

,business_id,category
0,0,Hotel
1,1,Hotel
2,2,Hotel
3,2,Group accommodation
4,2,Lodge


Ahora con los valores unicos se puede crear la tabla de categoria, hacer un join posteriormente y borrar la columna category de business.

In [138]:
category = business_category["category"].drop_duplicates().reset_index(drop=True) 
category = category.reset_index()
category.rename(columns={'index': 'category_id'}, inplace=True)
category

,category_id,category
0,0,Hotel
1,1,Group accommodation
2,2,Lodge
3,3,Motel
4,4,Resort hotel
...,...,...
590,590,Low income housing program
591,591,Boat club
592,592,Scrapbooking store
593,593,Rehabilitation center


In [139]:
business_category = business_category.merge(category, on="category").sort_values(by="business_id").drop(columns="category").reset_index(drop=True)
business_category.head()

,business_id,category_id
0,0,0
1,1,0
2,2,0
3,2,3
4,2,1


In [140]:
state_names = ["Alabama", "Alaska", "Arizona", "Arkansas", "California", "Colorado", "Connecticut", "Washington DC", "Delaware", "Florida", "Georgia",
               "Hawaii", "Idaho", "Illinois", "Indiana", "Iowa", "Kansas", "Kentucky", "Louisiana", "Maine", "Maryland", "Massachusetts",
               "Michigan", "Minnesota", "Mississippi", "Missouri", "Montana", "Nebraska", "Nevada", "New Hampshire", "New Jersey", "New Mexico",
               "New York", "North Carolina", "North Dakota", "Ohio", "Oklahoma", "Oregon", "Pennsylvania", "Rhode Island", "South Carolina",
               "South Dakota", "Tennessee", "Texas", "Utah", "Vermont", "Virginia", "Washington", "West Virginia", "Wisconsin", "Wyoming"]

In [143]:
states = pd.DataFrame()
states["state"] = valid_state_codes
states["state_name"] = state_names

### Users

In [145]:
reviews.head()

,user_id,name_x,time,rating,text,gmap_id
0,114992108396100335559,Cherena Dukes,1574336225144,4,Nice homes in fairly new subdivisions.,0x88892dc32d67742d:0x91dc8f0e85b37f5e
1,109682539851735063549,Lee Morrison,1543167503668,2,Roads are terrible. Neighbors park in yards. ...,0x88892dc32d67742d:0x91dc8f0e85b37f5e
26,112665186528292597385,Charlie Cline,1454821094934,4,Good location. Nice staff. Clean. Breakfast ar...,0x88890491e9416625:0x2f85307a06ddb16
27,108637957038201408174,Jennifer Passmore,1491966256419,5,Super clean very friendly,0x88890491e9416625:0x2f85307a06ddb16
28,109079098695645726629,TheBeckyboo926,1476494074021,5,Amazing! We had a conference room and it was a...,0x88890491e9416625:0x2f85307a06ddb16


In [161]:
users = reviews[["user_id", "name_x"]].drop_duplicates(subset=['user_id', 'name_x']).reset_index(drop=True)

In [162]:
users.duplicated(subset="user_id").sum()

16

In [163]:
users.drop_duplicates(subset="user_id", inplace=True)

In [164]:
users.rename(columns={'user_id': 'google_id'}, inplace=True)

In [165]:
users = users.reset_index(drop=True)

In [166]:
users = users.reset_index()
users.rename(columns={'index': 'user_id', 'name_x': 'user_name'}, inplace=True)
users

,user_id,google_id,user_name
0,0,114992108396100335559,Cherena Dukes
1,1,109682539851735063549,Lee Morrison
2,2,112665186528292597385,Charlie Cline
3,3,108637957038201408174,Jennifer Passmore
4,4,109079098695645726629,TheBeckyboo926
...,...,...,...
43737,43737,111182362803426279975,Matthew Kohl
43738,43738,105737959930315802253,Fred Rosa
43739,43739,113801734532377408633,Robin Reed
43740,43740,117464169380525203120,Jorden Parks


## Tabla de hechos

In [168]:
reviews['time'] = pd.to_datetime(reviews['time'], unit='ms')
reviews['time'] = reviews['time'].dt.strftime('%Y-%m-%d')

In [169]:
reviews.drop(columns="name_x", inplace=True)

In [172]:
reviews.rename(columns={'time': 'date', 'user_id': 'google_id',
                        'text': 'review'}, inplace=True)

In [193]:
reviews = reviews.sort_values(by="date").reset_index(drop=True)

In [194]:
reviews = reviews.reset_index() 
reviews.rename(columns={'index': 'review_id'}, inplace=True)
reviews

,review_id,google_id,date,rating,review,gmap_id
0,0,103650800402109980585,2007-09-19,1,"Called on Saturday, would not see us even thou...",0x880f0cae93214435:0x328a1fd2d302c34f
1,1,107032311994563884394,2007-12-30,4,We stayed 3 nights - Thanksgiving weekend 2007...,0x88646e55c2098b83:0x2ec91f6e70b68e3e
2,2,109828460192402399885,2008-02-11,3,too many tourists.,0x89c258f5922d74b9:0x85af792fa088fefb
3,3,113399057748819148056,2008-05-24,5,"Great Security , service, and accomodation. P...",0x8640bfeb782384ab:0x2b855859e2ca69c2
4,4,116437168519004876189,2008-09-20,3,"I'm pretty easy to please. So, I loved the clo...",0x883a4684edffecdb:0x92acc61b87cd2de9
...,...,...,...,...,...,...
44752,44752,101740175963128176096,2021-08-27,3,Kool\nGood enough,0x8824d6e1fe1e747f:0xc8095268889e2428
44753,44753,103615591642852508667,2021-08-28,5,My room was 100% updated and spotless! Absolut...,0x890069b8ea819cb1:0xd7a54f4a9948981e
44754,44754,115210921748709918356,2021-08-31,1,Terrible,0x88626b3eef6314db:0x1c0b19d1d69d1d24
44755,44755,104664581384426348152,2021-09-03,5,"Stunning view, cute and extremely clean little...",0x54907fca2751e187:0x3d6ae31ac18da483


## Guardando dataset

In [195]:
# Exportamos
reviews.to_csv('../processed_data/reviews_google_normalizado.csv', index=False, sep='|')

In [178]:
users.to_csv('../processed_data/users_google.csv', index=False, sep='|')

In [179]:
business.to_csv('../processed_data/business_google.csv', index=False, sep='|')

In [180]:
category.to_csv('../processed_data/category_google.csv', index=False, sep='|')

In [181]:
business_category.to_csv('../processed_data/business_category_google.csv', index=False, sep='|')

In [182]:
states.to_csv('../processed_data/states.csv', index=False, sep='|')

Para efectos practicos de exploracion de datos y modelado, se guarda tambien una tabla no normalizada.

In [197]:
reviews_merge = reviews.merge(business, on="gmap_id")

In [198]:
reviews_merge.to_csv('../processed_data/reviews_google_sin_normalizar.csv', index=False, sep='|')